In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## Outline

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

## 1. Common Element

#### Outline

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

#### Function

In [2]:
def convert_common(d):
    return {
            'Region'                     : d['Common']['Region'],
            'IsSimplifiedInput'          : d['Common']['IsSimplifiedInput'],
            'MainOccupantRoomFloorArea'  : d['Common']['MainOccupantRoomFloorArea'],
            'OtherOccupantRoomFloorArea' : d['Common']['OtherOccupantRoomFloorArea'],
            'TotalFloorArea'             : d['Common']['TotalFloorArea']
    }

## 2. Walls Element

#### Outline

'Walls'要素はそのまま維持する。

#### Function

In [3]:
def make_layer(name, thick, cond, specH, **kwargs) :
    
    return {
            'name'  : name,
            'thick' : thick,
            'cond'  : cond,
            'specH' : specH
    }

In [4]:
def make_parts(InputMethod, Parts, type = None, **kwargs) :
    
    parts_copy = []
    
    for x in Parts :
        part = {}
        if InputMethod == 'InputAllDetails' :
            part['AreaRatio'] = x['AreaRatio']
        elif InputMethod == 'InputAllLayers' :
            if type == 'Ceiling' :
                part['TypeRoof'] = x['TypeRoof']
            elif type == 'Wall' or type == 'BoundaryWall' :
                part['TypeWall'] = x['TypeWall']
            elif type == 'Floor' or type == 'BoundaryFloor' :
                part['TypeFloor'] = x['TypeFloor']
            elif type == 'BoundaryCeiling' :
                part['TypeCeiling'] = x['TypeCeiling']
            else :
                raise ValueError       
            
        part['Layers'] = [make_layer(**layer) for layer in x['Layers']]
        parts_copy.append(part)
        
    return parts_copy

In [62]:
def make_wall(IsSimplifiedInput, name, direction, area, space, type, structure, IsSunshadeInput, 
              UA = None, IsInContactWithOutsideAir = None, InputMethod = None,
              RoofConstructionMethod = None, WallConstructionMethod = None, FloorConstructionMethod = None, CeilingConstructionMethod = None, 
              TypeFloor = None, TypeWall = None, TypeCeiling = None,TypeRoof = None, ConstructionMethod = None, 
              RoofInsulationMethod = None, WallInsulationMethod = None, Radding = None, Parts = None,
              Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    wall = {
        'name':      name,
        'direction': direction,
        'area':      area,
        'space':     space,
        'type':      type,
        'structure':     structure,        
        'IsSunshadeInput': IsSunshadeInput        
    }
    
    # 簡易外皮入力の場合
    if IsSimplifiedInput == True :
            wall['UA'] = UA 
            
    # 簡易外皮入力でない場合
    else :
        wall['InputMethod'] = InputMethod
        if structure == 'wood' :
            if InputMethod == 'InputUA' :
                wall['UA'] = UA 
            elif InputMethod == 'InputAllDetails' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            elif InputMethod == 'InputAllLayers' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts, type = type)
                if type == 'Ceiling' :
                    wall['RoofConstructionMethod'] = RoofConstructionMethod
                elif type == 'Wall' or type == 'BoundaryWall' :
                    wall['WallConstructionMethod'] = WallConstructionMethod
                elif type == 'Floor' or type == 'BoundaryFloor' :
                    wall['FloorConstructionMethod'] = FloorConstructionMethod
                elif type == 'BoundaryCeiling' :
                    wall['CeilingConstructionMethod'] = CeilingConstructionMethod
                else :
                    raise ValueError               
            elif InputMethod == 'InputUR' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                if type == 'Floor' or type == 'BoundaryCeiling' or type == 'BoundaryFloor' :
                    wall['ConstructionMethod'] = ConstructionMethod
                elif type == 'Wall' or type == 'BoundaryWall' :
                    wall['WallInsulationMethod'] = WallInsulationMethod
                    if WallInsulationMethod == 'InsulColumn' or WallInsulationMethod == 'InsulColumnandadd' :
                        wall['ConstructionMethod'] = ConstructionMethod       
                elif type == 'Ceiling' :
                    pass
                elif type == 'Roof' :    
                    wall['RoofInsulationMethod'] = RoofInsulationMethod                    
                else :
                    raise ValueError 
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError                    
        elif structure == 'RC' :
            if InputMethod == 'InputUA' :
                wall['UA'] = UA 
            elif InputMethod == 'InputLayers' :
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError            
        elif structure == 'steel' :
            if InputMethod == 'InputUA' :
                wall['UA'] = UA 
            elif InputMethod == 'InputUR' :
                wall['Radding'] = Radding
                wall['IsInContactWithOutsideAir'] = IsInContactWithOutsideAir
                wall['Parts'] = make_parts(InputMethod = InputMethod, Parts = Parts)
            else :
                raise ValueError        
        elif structure == 'other' :
            pass
        else :
            raise ValueError        

    # 日除けの設定
    if IsSunshadeInput == True :
        wall['Y1'] = Y1
        wall['Y2'] = Y2
        wall['Z'] = Z
 
    return wall

In [6]:
def convert_wall( d ):    
    return [ make_wall(IsSimplifiedInput = d['Common']['IsSimplifiedInput'], **f) for f in d['Walls'] ]

## 3. Windows Element

#### Outline

'Windows'要素はそのまま維持する。

#### Function

In [7]:
def make_window(IsSimplifiedInput, name, direction, area, space, IsSunshadeInput, UW, TypeWindow = None, IsEtaValueInput = None,
                EtaCooling = None, EtaHeating = None, Eta = None,TypeFrame = None, TypeGlass = None, TypeShade = None, 
                EtaInside = None,TypeFrameInside = None, TypeGlassInside = None, TypeShadeInside = None, 
                EtaOutside = None,TypeFrameOutside = None, TypeGlassOutside = None, TypeShadeOutside = None, 
                Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    window = {
        'name':      name,
        'direction': direction,
        'area':      area,
        'space':     space,
        'UW':         UW,
        'IsSunshadeInput': IsSunshadeInput        
    } 
    
    # 簡易外皮入力の場合
    if IsSimplifiedInput == True :
            window['EtaCooling'] = EtaCooling 
            window['EtaHeating'] = EtaHeating    
            
    # 簡易外皮入力でない場合
    else :
        window['TypeWindow'] = TypeWindow
        window['IsEtaValueInput'] = IsEtaValueInput
        # 一重窓
        if TypeWindow == 'Single' :
            if IsEtaValueInput == True :
                window['Eta'] = Eta
            else :
                window['TypeFrame'] = TypeFrame
                window['TypeGlass'] = TypeGlass
                window['TypeShade'] = TypeShade
        # 二重窓
        elif TypeWindow == 'Double' :
            if IsEtaValueInput == True :
                window['EtaInside'] = EtaInside
                window['EtaOutside'] = EtaOutside
            else :
                window['TypeFrameInside'] = TypeFrameInside
                window['TypeGlassInside'] = TypeGlassInside
                window['TypeShadeInside'] = TypeShadeInside
                window['TypeFrameOutside'] = TypeFrameOutside
                window['TypeGlassOutside'] = TypeGlassOutside
                window['TypeShadeOutside'] = TypeShadeOutside
        else :
            raise ValueError        
    
    # 日除けの設定
    if IsSunshadeInput == True :
        # 一重窓
        if TypeWindow == 'Single' :
                window['TypeGlass'] = TypeGlass
        # 二重窓
        elif TypeWindow == 'Double' :
                window['TypeGlassInside'] = TypeGlassInside
                window['TypeGlassOutside'] = TypeGlassOutside
        else :
            raise ValueError        
                
        window['Y1'] = Y1
        window['Y2'] = Y2
        window['Z'] = Z

    return window

In [8]:
def convert_window( d ):    
    return [ make_window(IsSimplifiedInput = d['Common']['IsSimplifiedInput'], **f) for f in d['Windows'] ]

## 4. Doors Element

#### Outline

'Doors'要素はそのまま維持する。

#### Function

In [9]:
def make_door(name, direction, area, space, U, IsSunshadeInput, Y1 = None, Y2 = None, Z = None, **kwargs ):

    door = {
        'name':      name,
        'direction': direction,
        'area':      area,
        'space':     space,
        'U':         U,
        'IsSunshadeInput': IsSunshadeInput
    }
    
    # 日除けの設定
    if IsSunshadeInput == True :
        door['Y1'] = Y1
        door['Y2'] = Y2
        door['Z'] = Z
    
    return door

In [10]:
def convert_door( d ):    
    return [ make_door(**f) for f in d['Doors'] ]

## 5. Heatbridges Element

#### Outline

'Heatbridges'要素はそのまま維持する。

#### Function

In [11]:
def make_heatbridge(name, structure, direction1, direction2, space, length, psi):
    return {
        'name':       name,
        'structure':  structure,
        'direction1': direction1,
        'direction2': direction2,
        'length':     length,
        'space':      space,
        'psi':        psi
    }

In [12]:
def convert_heatbridge( d ):    
    return [ make_heatbridge(**f) for f in d['Heatbridges'] ]

## 6. EarthfloorPerimeters Element

#### Outline

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [13]:
def make_earthfloorperimeter( direction, length, name, psi, space):
    return {
        'direction': direction,
        'length':    length,
        'name':      name,
        'psi':       psi,
        'space':     space
    }

In [14]:
def convert_earthfloorperimeter( d ):    
    return [ make_earthfloorperimeter(**f) for f in d['EarthfloorPerimeters'] ]

## 7. Earthfloors Element

#### Outline

'Earthfloors'要素はそのまま維持する。

#### Function

In [15]:
def make_earthfloor( name, area, space ):
    return {
        'name': name,
        'area': area,
        'space': space
    }

In [16]:
def convert_earthfloor( d ):        
    return [ make_earthfloor(**f) for f in d['Earthfloors'] ]

## 8. InnerWalls Element

## 8.1. 用途別床面積を取得する関数

In [17]:
def get_area_floor_tuple(d_common):
    return d_common['MainOccupantRoomFloorArea'], \
           d_common['OtherOccupantRoomFloorArea'], \
           d_common['TotalFloorArea'] - d_common['MainOccupantRoomFloorArea'] - d_common['OtherOccupantRoomFloorArea']

In [18]:
def get_area_floor_dictionary(d_common):
    return { 'main': d_common['MainOccupantRoomFloorArea'],
             'other': d_common['OtherOccupantRoomFloorArea'], \
             'nonliving': d_common['TotalFloorArea'] - d_common['MainOccupantRoomFloorArea'] - d_common['OtherOccupantRoomFloorArea']
            }

## 8.2. InnerWalls Element - Ground Floor 

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [19]:
def get_inner_floor_layers():
    return [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}]

### 2) 床下空間上の床面積の推定

#### Outline  
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。  
- 室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。

$\sum_{i}^{}A_{b,i}>0$ かつ $A_{dif,MR} + A_{dif,OR} + A_{dif,NR}>0$ の場合

$$
\displaystyle
A_{f,MR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,MR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,OR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,OR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,NR}= \sum_{i}^{}A_{b,i} \times \frac{ A_{dif,NR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>  
　　 ($\sum_{i}^{}A_{b,i}≦A_{dif,MR} + A_{dif,OR} + A_{dif,NR}$を想定)  
$A_{f,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_f$: 室と床下空間の間の床面積の合計, m<sup>2</sup>  
$A_{dif,MR}$: 主たる居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,OR}$: その他の居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  
$A_{dif,NR}$: 非居室において、方位が「下面」の部位の面積と土間床等の面積の合計と、床面積の差, m<sup>2</sup>  

$$
\displaystyle
A_{dif, MR} = max(0, A_{MR} - \sum_i A_{of,MR,i} - \sum_i A_{ef,MR,i})
$$

$$
\displaystyle
A_{dif, OR} = max(0, A_{OR} - \sum_i A_{of,OR,i} - \sum_i A_{ef,OR,i})
$$

$$
\displaystyle
A_{dif, NR} = max(0, A_{NR} - \sum_i A_{of,NR,i} - \sum_i A_{ef,NR,i})
$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>  

#### Function

In [20]:
# 方位が「下面」である外皮の部位の面積の合計
def get_A_of_bottom(d, s):
    # s : space name

    l = []
    for x in ['Walls','Windows','Doors'] :
        if (x in d) == True :
            l = l + d[x]           
            
    return sum( y['area'] for y in l if y['space'] == s and y['direction'] == 'bottom' )

In [21]:
# 土間床等の面積の合計
def get_A_of_EarthFloor(d, s):
    # s : space name

    if ('Earthfloors' in d ) == True :
        return sum( y['area'] for y in d['Earthfloors'] if y['space'] == s )
    else :
        return 0

In [22]:
# 方位が「下面」である外皮の部位と土間床等の面積の合計
def get_A_of(d, s):        
    return get_A_of_bottom(d, s) + get_A_of_EarthFloor(d, s)

In [23]:
# 床下空間に属する土間床等の面積の取得
def get_A_b(d):
    if ('Earthfloors' in d ) == True :
        return sum(earthfloor['area'] for earthfloor in d['Earthfloors'] if earthfloor['space'] == 'underfloor')
    else :
        return 0

#### Function 1

In [24]:
def get_area_floor_over_underfloor_tuple(d):
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 下面（direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_of_MR, A_of_OR, A_of_NR = get_A_of(d, 'main'), get_A_of(d, 'other'), get_A_of(d, 'nonliving')
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    A_dif_MR, A_dif_OR, A_dif_NR = max(0.0, A_MR - A_of_MR), max(0.0, A_OR - A_of_OR), max(0.0, A_NR - A_of_NR)

    A_dif = A_dif_MR + A_dif_OR + A_dif_NR

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)
    
    # 室と床下空間の間の床面を設定する場合
    if A_b > 0 and A_dif > 0 :    
        # 居室と床下空間との間の床の面積
        A_f_MR, A_f_OR, A_f_NR = A_b * A_dif_MR / A_dif, A_b * A_dif_OR / A_dif, A_b * A_dif_NR / A_dif
        return True, A_f_MR, A_f_OR, A_f_NR
    
    # 室と床下空間の間の床面を設定しない場合
    else :
        return False, None, None, None

#### Function 2

In [25]:
def get_area_floor_over_underfloor_dictionary(d):

    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 下面（direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_of= {x : get_A_of(d, x) for x in ['main', 'other', 'nonliving']}
    
    # 方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差
    A_dif = {x : max(0.0, A[x] - A_of[x]) for x in ['main', 'other', 'nonliving']}
    
    A_dif_sum = sum(A_dif[x] for x in A_dif.keys())

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)

    #  室と床下空間の間の床面積の推定
    A_f = min(A_dif_sum, A_b)    

    # 室と床下空間の間の床面を設定する場合
    if A_b > 0 and A_dif_sum > 0 :    
        # 居室と床下空間との間の床の面積
        A_f = {x : A_b * A_dif[x] / A_dif_sum for x in ['main', 'other', 'nonliving']}
        return True, A_f

    # 室と床下空間の間の床面を設定しない場合
    else :
        return False, None

#### Example

In [26]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_floor_over_underfloor_tuple(d))
print(get_area_floor_over_underfloor_dictionary(d))

(True, 0.0, 6.666666666666667, 3.3333333333333335)
(True, {'main': 0.0, 'other': 6.666666666666667, 'nonliving': 3.3333333333333335})


### 3) 室と床下空間の間の床の構築

#### Function 1

In [27]:
def get_innerwall_groundfloor_tuple(d):
    
    # 室と床下空間の間の床面積の推定
    IsInnerwallGroundfloor, A_if_MR, A_if_OR, A_if_NR = get_area_floor_over_underfloor_tuple(d)
    
    # 室と床下空間の間の床面を設定する場合
    if IsInnerwallGroundfloor == True :    
        # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
        ifs = []

        if A_if_MR > 0 :
            ifs.extend([{
                'name' : 'GroundFloor_Main',
                'type' : 'GroundFloor',
                'direction' : 'Horizontal',
                'area' : A_if_MR,
                'Layers' : get_inner_floor_layers(),
                'spaces' : ['main','underfloor'],
                }])

        if A_if_OR > 0 :
            ifs.extend([ {
                'name' : 'GroundFloor_Other',
                'type' : 'GroundFloor',
                'direction' : 'Horizontal',
                'area' : A_if_OR,
                'Layers' : get_inner_floor_layers(),
                'spaces' : ['other','underfloor'],
                }])    

        if A_if_NR > 0 :
            ifs.extend([{
                'name' : 'GroundFloor_Nonliving',
                'type' : 'GroundFloor',
                'direction' : 'Horizontal',
                'area' : A_if_NR,
                'Layers' : get_inner_floor_layers(),
                'spaces' : ['nonliving','underfloor'],
                }])        
        
        return ifs
    # 室と床下空間の間の床面を設定しない場合
    else :
        return []

#### Function 2

In [28]:
def get_innerwall_groundfloor_dictionary(d):
    
    # 室と床下空間の間の床面積の推定
    IsInnerwallGroundfloor, areas_innerfloor = get_area_floor_over_underfloor_dictionary(d)
        
    # 室と床下空間の間の床面を設定する場合
    if IsInnerwallGroundfloor == True :    
        # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
        ifs = []
        
        for space in ['main', 'other', 'nonliving'] :
            if areas_innerfloor[space] > 0 :
                ifs.extend([ {
                    'name' : 'GroundFloor_' + space,
                    'type' : 'GroundFloor',
                    'direction' : 'Horizontal',
                    'area' : areas_innerfloor[space],
                    'Layers' : get_inner_floor_layers(),
                    'spaces' : [space,'underfloor'],}])

        return ifs
    # 室と床下空間の間の床面を設定しない場合
    else :
        return []

## 8.3. InnerWalls Element - Inner Floor

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [29]:
def get_inner_floor_layers():
    return [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}]

### 2) 室上の床面積の推定

#### Outline

$$
\displaystyle
A_{if,MR}= \max \left(0, A_{MR} -  \left(A_{gf,MR} + \sum_i A_{of,MR,i} + \sum_i A_{ef,MR,i} \right)\right)
$$

$$
\displaystyle
A_{if,OR}= \max \left(0, A_{OR} -  \left(A_{gf,OR} + \sum_i A_{of,OR,i} + \sum_i A_{ef,OR,i} \right)\right)
$$

$$
\displaystyle
A_{if,NR}= \max \left(0, A_{NR} -  \left(A_{gf,NR} + \sum_i A_{of,NR,i} + \sum_i A_{ef,NR,i} \right)\right)
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{gf,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{gf,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{ef,MR,i}$: 主たる居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$: その他の居室の土間床$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$: 非居室の土間床$i$の面積, m<sup>2</sup>   

#### Function

In [30]:
# 床下空間上の床の面積の合計
def get_A_if_GroundFloor(innerwalls, s):
    # s : space name

    return sum( y['area'] for y in innerwalls if (y['type'] == 'GroundFloor'and y['spaces'] == [s,'underfloor']))

In [31]:
# 床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の面積の合計
def get_A_f(d, s, innerwalls):
    # s : space name

    return get_A_if_GroundFloor(innerwalls, s) + get_A_of_bottom(d, s) + get_A_of_EarthFloor(d, s)

#### Function 1

In [32]:
def get_area_floor_over_room_tuple(d, innerwalls):
       
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_f_MR, A_f_OR, A_f_NR = get_A_f(d, 'main', innerwalls), get_A_f(d, 'other', innerwalls), get_A_f(d, 'nonliving', innerwalls)
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    A_if_MR, A_if_OR, A_if_NR = max(0.0, A_MR - A_f_MR), max(0.0, A_OR - A_f_OR), max(0.0, A_NR - A_f_NR)
    
    return A_if_MR, A_if_OR, A_if_NR

#### Function 2

In [33]:
def get_area_floor_over_room_dictionary(d, innerwalls):
    
    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 床下空間上の床面積と下面（Direction:Bottom）の壁・窓・ドア面積と土間床等の面積の合計値の取得
    A_f = {x : get_A_f(d, x, innerwalls) for x in ['main', 'other', 'nonliving']}
    
    # 床下空間上の床面積と方位が「下面」の部位の面積と土間床等の面積の合計値と、床面積の差を、室上の床面積とみなす。
    A_if = {x : max(0.0, A[x] - A_f[x]) for x in ['main', 'other', 'nonliving']}
    
    return A_if

#### Example

In [34]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_floor_over_room_tuple(d, innerwalls))
print(get_area_floor_over_room_dictionary(d, innerwalls))

(0.0, 45.0, 30.0)
{'main': 0.0, 'other': 45.0, 'nonliving': 30.0}


### 3) 室下の天井面積の推定

#### Outline

$$
\displaystyle
A_{ic,MR}= \max \left(0, A_{MR} -  \sum_i A_{oc,MR,i} \right)
$$

$$
\displaystyle
A_{ic,OR}= \max \left(0, A_{OR} -  \sum_i A_{oc,OR,i} \right)
$$

$$
\displaystyle
A_{ic,NR}= \max \left(0, A_{NR} -  \sum_i A_{oc,NR,i} \right)
$$

$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{oc,MR,i}$: 方位が「上面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,OR,i}$: 方位が「上面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{oc,NR,i}$: 方位が「上面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  

#### Function

In [35]:
# 方位が「上面」である外皮の部位の面積の合計
def get_A_of_top(d, s):
    # s : space name

    d_top = []
    for x in ['Walls','Windows','Doors'] :
        if (x in d) == True :
            d_top = d_top + d[x]           
            
    return sum( y['area'] for y in d_bottom if y['space'] == s and y['direction'] == 'top' )

#### Function 1

In [36]:
# 方位が「下面」である外皮の部位の面積の合計
def get_A_of_top(d, s):
    # s : space name

    l = []
    for x in ['Walls','Windows','Doors'] :
        if (x in d) == True :
            l = l + d[x]           
            
    return sum( y['area'] for y in l if y['space'] == s and y['direction'] == 'top' )

In [37]:
def get_area_ceiling_below_room_tuple(d):
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    A_oc_MR, A_oc_OR, A_oc_NR = get_A_of_top(d, 'main'), get_A_of_top(d, 'other'), get_A_of_top(d, 'nonliving')
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    A_ic_MR, A_ic_OR, A_ic_NR = max(0.0, A_MR - A_oc_MR), max(0.0, A_OR - A_oc_OR), max(0.0, A_NR - A_oc_NR)
   
    return A_ic_MR, A_ic_OR, A_ic_NR

#### Function 2

In [38]:
def get_area_ceiling_below_room_dictionary(d):
    
    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
    
    # 上面（Direction:Top）の壁・窓・ドア面積の合計値の取得
    A_oc = {x : get_A_of_top(d, x) for x in ['main', 'other', 'nonliving']}
    
    # 方位が「上面」の部位の面積の合計値と、床面積の差
    A_ic = {x : max(0.0, A[x] - A_oc[x])for x in ['main', 'other', 'nonliving']}
   
    return A_ic

#### Example

In [39]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

print(get_area_ceiling_below_room_tuple(d))
print(get_area_ceiling_below_room_dictionary(d))

(0.0, 60.0, 30.0)
{'main': 0.0, 'other': 60.0, 'nonliving': 30.0}


### 4) 室間の床と天井の面積の推定

#### Outline

$$
\displaystyle
A_{if,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,OR}+A_{ic,NR}} \times A_{if,MR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,NR} = 0 ) \\
\frac{A_{ic,NR}}{A_{ic,MR}+A_{ic,NR}} \times A_{if,OR} & ( A_{ic,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{if,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,MR} = 0 ) \\
\frac{A_{ic,MR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{if,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{ic,OR} = 0 ) \\
\frac{A_{ic,OR}}{A_{ic,MR}+A_{ic,OR}} \times A_{if,NR} & ( A_{ic,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  

$$
\displaystyle
A_{ic,MR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,MR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,OR}+A_{if,NR}} \times A_{ic,MR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,OR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,OR,NR} = \left \{
\begin{array}{ll}
0 & ( A_{if,NR} = 0 ) \\
\frac{A_{if,NR}}{A_{if,MR}+A_{if,NR}} \times A_{ic,OR} & ( A_{if,NR} > 0 ) \\
\end{array}
\right.
$$

$$
\displaystyle
A_{ic,NR,MR} = \left \{
\begin{array}{ll}
0 & ( A_{if,MR} = 0 ) \\
\frac{A_{if,MR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,MR} > 0 ) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,NR,OR} = \left \{
\begin{array}{ll}
0 & ( A_{if,OR} = 0 ) \\
\frac{A_{if,OR}}{A_{if,MR}+A_{if,OR}} \times A_{ic,NR} & ( A_{if,OR} > 0 ) \\
\end{array}
\right.
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との、間の非居室の床の面積, m<sup>2</sup>  
$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

#### Function 1

In [40]:
def get_area_slab_between_rooms_tuple(d, innerwalls):
    
    def get_A_if_dictionary(s_m, s_n, A_if_m, A_ic_n, A_ic_o) :
        return ([
            {
                'name' : 'InnerFloor' + s_m + s_n,
                'area' : A_if_m * A_ic_n / (A_ic_n + A_ic_o),
                'space' : s_m,
                'nextspace' : s_n
            }
        ])
    
    def get_A_ic_dictionary(s_m, s_n, A_ic_m, A_if_n, A_if_o) :
        return ([
            {
                'name' : 'InnerCeiling' + s_m + s_n,
                'area' : A_ic_m * A_if_n / (A_if_n + A_if_o),
                'space' : s_m,
                'nextspace' : s_n
            }
        ])
    
    # 室上の床面積の取得
    A_if_MR, A_if_OR, A_if_NR = get_area_floor_over_room_tuple(d, innerwalls)
    
    # 室下の天井面積の取得
    A_ic_MR, A_ic_OR, A_ic_NR = get_area_ceiling_below_room_tuple(d)

    # 室上の床面積を、他の用途の室下の天井面積で案分する。
    A_innerfloor=[]
    if A_if_MR > 0 and A_ic_OR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('main', 'other', A_if_MR, A_ic_OR, A_ic_NR))
    if A_if_MR > 0 and A_ic_NR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('main', 'nonliving', A_if_MR, A_ic_NR, A_ic_OR))
    if A_if_OR > 0 and A_ic_MR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('other', 'main', A_if_OR, A_ic_MR, A_ic_NR))
    if A_if_OR > 0 and A_ic_NR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('other', 'nonliving', A_if_OR, A_ic_NR, A_ic_MR))
    if A_if_NR > 0 and A_ic_MR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('nonliving', 'main', A_if_NR, A_ic_MR, A_ic_OR))
    if A_if_NR > 0 and A_ic_OR > 0 :
        A_innerfloor.extend(get_A_if_dictionary('nonliving', 'other', A_if_NR, A_ic_OR, A_ic_MR))

    # 室下の天井面積を、他の用途の室下上の床面積で案分する。
    A_innerceiling=[]
    if A_ic_MR > 0 and A_if_OR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('main', 'other', A_ic_MR, A_if_OR, A_if_NR))
    if A_ic_MR > 0 and A_if_NR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('main', 'nonliving', A_ic_MR, A_if_NR, A_if_OR))
    if A_ic_OR > 0 and A_if_MR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('other', 'main', A_ic_OR, A_if_MR, A_if_NR))
    if A_ic_OR > 0 and A_if_NR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('other', 'nonliving', A_ic_OR, A_if_NR, A_if_MR))
    if A_ic_NR > 0 and A_if_MR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('nonliving', 'main', A_ic_NR, A_if_MR, A_if_OR))
    if A_ic_NR > 0 and A_if_OR > 0 :
        A_innerceiling.extend(get_A_ic_dictionary('nonliving', 'other', A_ic_NR, A_if_OR, A_if_MR))
  
    return {'InnerFloors' : A_innerfloor, 'InnerCeilings' : A_innerceiling}

#### Function 2

In [41]:
def get_area_slab_between_rooms_dictionary(d, innerwalls):
    
    space = ['main', 'other', 'nonliving']  
    
    # 室上の床面積の取得
    A_if =  get_area_floor_over_room_dictionary(d, innerwalls)
    
    # 室下の天井面積の取得
    A_ic =  get_area_ceiling_below_room_dictionary(d)

    # 室上の床面積を、他の用途の室下の天井面積で案分する。
    A_innerfloor = []
    for x in [['main', 'other', 'nonliving'], ['main', 'nonliving', 'other'], ['other', 'main', 'nonliving'], \
              ['other', 'nonliving', 'main'], ['nonliving', 'main', 'other'], ['nonliving', 'other', 'main']] :
        
        # 自室、隣接室、他の室の設定
        s_m, s_n, s_o = x[0], x[1], x[2]

        if A_if[s_m] > 0 and A_ic[s_n] > 0 :
            A_innerfloor.extend([
                {
                    'name' : 'InnerFloor' + s_m + s_n,
                    'area' : A_if[s_m] * A_ic[s_n] / (A_ic[s_n] + A_ic[s_o]),
                    'space' : s_m,
                    'nextspace' : s_n
                }
            ])

    # 室下の天井面積を、他の用途の室下上の床面積で案分する。
    A_innerceiling = []
    for x in [['main', 'other', 'nonliving'], ['main', 'nonliving', 'other'], ['other', 'main', 'nonliving'], \
              ['other', 'nonliving', 'main'], ['nonliving', 'main', 'other'], ['nonliving', 'other', 'main']] :
                
        # 自室、隣接室、他の室の設定
        s_m, s_n, s_o = x[0], x[1], x[2]
        
        if A_ic[s_m] > 0 and A_if[s_n] > 0 :
            A_innerceiling.extend([
                {
                    'name' : 'InnerCeiling' + s_m + s_n,
                    'area' : A_ic[s_m] * A_if[s_n] / (A_if[s_n] + A_if[s_o]),
                    'space' : s_m,
                    'nextspace' : s_n
                }
            ])
  
    return {'InnerFloors' : A_innerfloor, 'InnerCeilings' : A_innerceiling}

#### Example

In [42]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

innerwalls =  [
    { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['Main','Underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
    { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['Other','Underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_slab_between_rooms_tuple(d, innerwalls))
print(get_area_slab_between_rooms_dictionary(d, innerwalls))

{'InnerFloors': [{'name': 'InnerFloorothernonliving', 'area': 60.0, 'space': 'other', 'nextspace': 'nonliving'}, {'name': 'InnerFloornonlivingother', 'area': 30.0, 'space': 'nonliving', 'nextspace': 'other'}], 'InnerCeilings': [{'name': 'InnerCeilingothernonliving', 'area': 60.0, 'space': 'other', 'nextspace': 'nonliving'}, {'name': 'InnerCeilingnonlivingother', 'area': 30.0, 'space': 'nonliving', 'nextspace': 'other'}]}
{'InnerFloors': [{'name': 'InnerFloorothernonliving', 'area': 60.0, 'space': 'other', 'nextspace': 'nonliving'}, {'name': 'InnerFloornonlivingother', 'area': 30.0, 'space': 'nonliving', 'nextspace': 'other'}], 'InnerCeilings': [{'name': 'InnerCeilingothernonliving', 'area': 60.0, 'space': 'other', 'nextspace': 'nonliving'}, {'name': 'InnerCeilingnonlivingother', 'area': 30.0, 'space': 'nonliving', 'nextspace': 'other'}]}


### 5) 室間の床・天井の構築

#### Function

In [43]:
def get_innerwall_innerslab(d, innerwalls):
    
    # 室間の床・天井面積の推定
    areas_innerslab = get_area_slab_between_rooms_dictionary(d, innerwalls)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'name' : 'InnerFloor_' + x['space'],
        'type' : 'InnerFloor',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_layers(),
        'space' : x['space'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerFloors'] ])
    
    ifs.extend([ {
        'name' : 'InnerCeiling_' + x['space'],
        'type' : 'InnerCeiling',
        'direction' : 'Horizontal',
        'area' : x['area'],
        'Layers' : get_inner_floor_layers(),
        'space' : x['space'], 
        'nextspace' : x['nextspace'],
    } for x in areas_innerslab['InnerCeilings'] ])
        
    return ifs

## 8.5. InnerWalls Element - Inner Wall 

### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

#### Function

In [44]:
def get_inner_wall_layers():
    return [
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0},
        {'name': 'GPB', 'cond': 1/0.09, 'thick': 1,      'specH': 0},
        {'name': 'GPB', 'cond': 0.22,   'thick': 0.0125, 'specH': 830.0}
    ]

### 3) 内壁等面積の推定

#### Outline

用途間の内壁等の面積は、各用途の内壁等の面積より決定する。

$$
A_{iw,MR-OR} =\frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2}
$$

$$
A_{iw,MR-NR} = \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2}
$$

$$
A_{iw,NR-OR} = \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2}
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

各用途の内壁等の面積は、各用途の内壁等の面積の暫定値より、用途間の内壁等の面積が0㎡以上となる範囲で決定する。

$A_{iw,MR,temp}>A_{iw,OR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{OR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{OR}}{A_{OR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{NR}}{A_{OR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,MR} = A_{iw,MR,temp}
$$
$$
A_{iw,OR} = A_{iw,MR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,MR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,OR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,OR,temp}>A_{iw,MR,temp}+A_{iw,NR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{NR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{MR}}{A_{MR}+A_{NR}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{NR}}{A_{MR}+A_{NR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,OR} = A_{iw,OR,temp}
$$
$$
A_{iw,MR} = A_{iw,OR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$
$$
A_{iw,NR} = A_{iw,OR,temp}\times \frac{A_{iw,NR,temp}}{A_{iw,MR,temp}+A_{iw,NR,temp}}
$$

$A_{iw,NR,temp}>A_{iw,MR,temp}+A_{iw,OR,temp}$ の場合

　　$A_{MR}=0$ かつ $A_{OR}=0$ の場合

$$A_{iw,MR} = 0, A_{iw,OR} = 0, A_{iw,NR} = 0$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}=0$ かつ $A_{iw,NR,temp}=0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{MR}}{A_{MR}+A_{OR}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{OR}}{A_{MR}+A_{OR}}
$$

　　$A_{OR}>0$ または $A_{NR}>0$ で、 $A_{iw,OR,temp}>0$ または $A_{iw,NR,temp}>0$ の場合

$$
A_{iw,NR} = A_{iw,NR,temp}
$$
$$
A_{iw,MR} = A_{iw,NR,temp}\times \frac{A_{iw,MR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$
$$
A_{iw,OR} = A_{iw,NR,temp}\times \frac{A_{iw,OR,temp}}{A_{iw,MR,temp}+A_{iw,OR,temp}}
$$

上記以外の場合

$$A_{iw,MR} = A_{iw,MR,temp}, A_{iw,OR} = A_{iw,OR,temp}, A_{iw,NR} = A_{iw,NR,temp}$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{iw,MR,temp}$: 主たる居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,OR,temp}$: その他の居室の内壁等の面積の暫定値, m<sup>2</sup>  
$A_{iw,NR,temp}$: 非居室の内壁等の面積の暫定値, m<sup>2</sup>  

各用途の内壁等の面積の暫定値は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。

$$
A_{iw,MR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0,4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR,temp} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の鉛直面の外皮と内壁の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の鉛直面の外皮の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

#### Function

In [45]:
#  室用途別の内壁等の面積
def get_A_ow_for_space(d_ow, s):
    # s : space name       
    return sum(y['area'] for y in d_ow if (y['space'] == s and y['direction'] != 'top' and y['direction'] != 'bottom'))

In [46]:
# 内壁等の面積の暫定値の取得
def get_A_iw_temp(A_f, ar, A_ow):

    h = 2.8 # 階高, m

    return max(0.0, 4 * h * ar * A_f**0.5 - A_ow) if A_f > 0.0 else 0.0

#### Function 1

In [47]:
# 居室の外皮と床下空間との間の床の面積の合計を取得
def get_A_ow_tuple(d):

    # 室用途別の内壁等の面積
    d_ow=[]
    for y in ['Walls', 'Windows', 'Doors']:
        if (y in d) == True :
            d_ow.extend(d[y])

    A_ow_MR, A_ow_OR,A_ow_NR = get_A_ow_for_space(d_ow,'Main'), get_A_ow_for_space(d_ow,'Other'), get_A_ow_for_space(d_ow,'Nonliving')

    return A_ow_MR, A_ow_OR, A_ow_NR

In [48]:
# 内壁等の面積の取得
def get_A_iw_tuple(A_MR, A_OR, A_NR,A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp):

    if A_iw_space_MR_temp > A_iw_space_OR_temp + A_iw_space_NR_temp :
        if A_OR == 0 and A_NR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_OR_temp == 0 and A_iw_space_NR_temp == 0 :
            A_iw_space_MR = A_iw_space_MR_temp
            A_iw_space_OR = A_iw_space_MR_temp * A_OR/(A_OR + A_NR)
            A_iw_space_NR = A_iw_space_MR_temp * A_NR/(A_OR + A_NR)
        else :
            A_iw_space_MR = A_iw_space_MR_temp
            A_iw_space_OR = A_iw_space_MR_temp * A_iw_space_OR_temp/(A_iw_space_OR_temp + A_iw_space_NR_temp)
            A_iw_space_NR = A_iw_space_MR_temp * A_iw_space_NR_temp/(A_iw_space_OR_temp + A_iw_space_NR_temp)
            
    elif A_iw_space_OR_temp > A_iw_space_MR_temp + A_iw_space_NR_temp :
        if A_MR == 0 and A_NR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_MR_temp == 0 and A_iw_space_NR_temp == 0 :
            A_iw_space_OR = A_iw_space_OR_temp
            A_iw_space_MR = A_iw_space_OR_temp * A_MR/(A_MR + A_NR)
            A_iw_space_NR = A_iw_space_OR_temp * A_NR/(A_MR + A_NR)
        else :
            A_iw_space_OR = A_iw_space_OR_temp
            A_iw_space_MR = A_iw_space_OR_temp * A_iw_space_MR_temp/(A_iw_space_MR_temp + A_iw_space_NR_temp)
            A_iw_space_NR = A_iw_space_OR_temp * A_iw_space_NR_temp/(A_iw_space_MR_temp + A_iw_space_NR_temp)
            
    elif A_iw_space_NR_temp > A_iw_space_MR_temp + A_iw_space_OR_temp :
        if A_MR == 0 and A_OR == 0 :
            A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = 0, 0, 0
        elif A_iw_space_MR_temp == 0 and A_iw_space_OR_temp == 0 :
            A_iw_space_NR = A_iw_space_NR_temp
            A_iw_space_MR = A_iw_space_NR_temp * A_MR/(A_MR + A_OR)
            A_iw_space_OR = A_iw_space_NR_temp * A_OR/(A_MR + A_OR)
        else :
            A_iw_space_NR = A_iw_space_NR_temp
            A_iw_space_MR = A_iw_space_NR_temp * A_iw_space_MR_temp/(A_iw_space_MR_temp + A_iw_space_OR_temp)
            A_iw_space_OR = A_iw_space_NR_temp * A_iw_space_OR_temp/(A_iw_space_MR_temp + A_iw_space_OR_temp)
            
    else :
        A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp

    return A_iw_space_MR, A_iw_space_OR, A_iw_space_NR

In [49]:
# 室用途間の内壁等の面積の推定
# 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
def get_A_iw_cor_tuple(A_iw_min, A_iw_s1, A_iw_s2, A_iw_os):
    return max(A_iw_min, (A_iw_s1 + A_iw_s2 - A_iw_os)/2)

In [50]:
def get_area_inner_wall_tuple(d):
    
    # 周長を求める際の係数, -
    ar_MR, ar_OR, ar_NR = 1.2, 1.4, 1.4

    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor_tuple(d['Common'])
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow_MR, A_ow_OR, A_ow_NR = get_A_ow_tuple(d)

    # 内壁等の面積の暫定値の取得
    A_iw_space_MR_temp = get_A_iw_temp(A_MR, ar_MR, A_ow_MR)
    A_iw_space_OR_temp = get_A_iw_temp(A_OR, ar_OR, A_ow_OR)
    A_iw_space_NR_temp = get_A_iw_temp(A_NR, ar_NR, A_ow_NR)

    # 内壁等の面積の取得
    A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = get_A_iw_tuple(A_MR, A_OR, A_NR, A_iw_space_MR_temp, A_iw_space_OR_temp, A_iw_space_NR_temp)
    
    # 室間の内壁等の面積の最小値, m2
    A_iw_min_MR_to_OR, A_iw_min_MR_to_NR, A_iw_min_OR_to_NR = 0.0, 0.0, 0.0
    
    # 室用途間の内壁等の面積の推定
    return get_A_iw_cor_tuple(A_iw_min_MR_to_OR, A_iw_space_MR, A_iw_space_OR, A_iw_space_NR), \
           get_A_iw_cor_tuple(A_iw_min_MR_to_NR, A_iw_space_MR, A_iw_space_NR, A_iw_space_OR), \
           get_A_iw_cor_tuple(A_iw_min_OR_to_NR, A_iw_space_OR, A_iw_space_NR, A_iw_space_MR)

#### Function 2

In [51]:
# 居室の外皮と床下空間との間の床の面積の合計を取得
def get_A_ow_dictionary(d):

    # 室用途別の内壁等の面積
    d_ow=[]
    for y in ['Walls', 'Windows', 'Doors']:
        if (y in d) == True :
            d_ow.extend(d[y])

    return { x: get_A_ow_for_space(d_ow, x) for x in ['main', 'other', 'nonliving']}

In [52]:
# 内壁等の面積の取得
def get_A_iw_dictionary(A, A_iw_space_temp):

    A_sum = sum(A[x] for x in A_iw_space_temp.keys())
    A_iw_space_temp_sum = sum(A_iw_space_temp[x] for x in A_iw_space_temp.keys())
    
    A_iw_space = {x:A_iw_space_temp[x] for x in A_iw_space_temp.keys()}
    
    for x in ['main', 'other', 'nonliving'] :   
        
        # 用途間の内壁等の面積が0㎡以上となるよう、各用途の内壁等の面積を調整
        if 2*A_iw_space_temp[x] > A_iw_space_temp_sum :
            
            # 他の2用途の床面積が0㎡の場合は、全用途の内壁等の面積を0㎡とする
            if A[x] == A_sum :
                A_iw_space = {y:0 for y in ['main', 'other', 'nonliving']}
                
            # 他の2用途の内壁等の面積の暫定値が0㎡の場合は、他の2用途の内壁等の面積を、用途別床面積に応じて決定
            elif A_iw_space_temp[x] == A_iw_space_temp_sum :
                for y in ['main', 'other', 'nonliving'] :
                    if x == y :
                        A_iw_space[y] = A_iw_space_temp[y]
                    else :
                        A_iw_space[y] = A_iw_space_temp[x] * A[y]/(A_sum - A[x])
                        
            # 他の2用途の内壁等の面積を、内壁等の面積の暫定値に応じて決定            
            else :
                for y in ['main', 'other', 'nonliving'] :
                    if x == y :
                        A_iw_space[y] = A_iw_space_temp[y]
                    else :
                        A_iw_space[y] = A_iw_space_temp[x] * A_iw_space_temp[y]/(A_iw_space_temp_sum - A_iw_space_temp[x])
                
    return A_iw_space

In [53]:
# 室用途間の内壁等の面積の推定
# 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
def get_A_iw_cor_dictionary(A_iw_min, A_iw_s1, A_iw_s2, A_iw_sum):
    return max(A_iw_min, (2*(A_iw_s1 + A_iw_s2) - A_iw_sum)/2)

In [54]:
def get_area_inner_wall_dictionary(d, c):
    # c : correspondence of spaces

    # 周長を求める際の係数, -
    ar = {'main': 1.2, 'other': 1.4, 'nonliving': 1.4}

    # 用途別床面積の取得
    A = get_area_floor_dictionary(d['Common'])
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow = get_A_ow_dictionary(d)

    # 内壁等の面積の取得
    A_iw_space_temp = { x: get_A_iw_temp(A[x], ar[x], A_ow[x]) for x in ['main', 'other', 'nonliving']}
    A_iw_space = get_A_iw_dictionary(A, A_iw_space_temp)
    A_iw_space_sum = sum(A_iw_space[x] for x in A_iw_space.keys())

    # 室間の内壁等の面積の最小値, m2
    A_iw_min = {'main to other': 0.0, 'main to nonliving': 0.0, 'other to nonliving': 0.0}
    
    # 室用途間の内壁等の面積の推定
    return { x: get_A_iw_cor_dictionary(A_iw_min[x], A_iw_space[c[x][0]], A_iw_space[c[x][1]], A_iw_space_sum) for x in c.keys()}

#### Example

In [55]:
correspondence = {
    'main to other': ['main', 'other'],
    'main to nonliving': ['main', 'nonliving'],
    'other to nonliving': ['other', 'nonliving'] 
}  

d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

print(get_area_inner_wall_tuple(d))
print(get_area_inner_wall_dictionary(d, correspondence))

(54.59388622447443, 19.020025504219888, 66.86287151259015)
{'main to other': 54.59388622447443, 'main to nonliving': 19.020025504219888, 'other to nonliving': 66.86287151259015}


### 3) 内壁の構築

#### Function 1

In [56]:
def get_innerwall_wall_tuple(d):

    # 内壁面積の推定
    A_iw_MR_OR, A_iw_MR_NR, A_iw_OR_NR = get_area_inner_wall_tuple(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    if A_iw_MR_OR > 0 :
        iws.extend([{
            'name' : 'Innerwall_main to other',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : A_iw_MR_OR,
            'spaces' : ['main', 'other']
        }])

    if A_iw_MR_NR > 0 :
        iws.extend([{
            'name' : 'Innerwall_main to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : A_iw_MR_NR,
            'spaces' : ['main', 'nonliving']
        }])

    if A_iw_OR_NR > 0 :
        iws.extend([{
            'name' : 'Innerwall_other to nonliving',
            'type' : 'InnerWall',
            'direction' : 'Vertical',
            'Layers' : get_inner_wall_layers(),
            'area' : A_iw_OR_NR,
            'spaces' : ['other', 'nonliving']
        }])

    return iws

#### Function 2

In [57]:
def get_innerwall_wall_dictionary(d):

    # 用途と複数用途間の対応関係の定義
    correspondence = {
        'main to other': ['main', 'other'],
        'main to nonliving': ['main', 'nonliving'],
        'other to nonliving': ['other', 'nonliving'] 
    }  

    # 内壁面積の推定
    areas_innerwall = get_area_inner_wall_dictionary(d, correspondence)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    for c in correspondence.keys() :
        if areas_innerwall[c] > 0 :
            iws.extend([ {
                'name' : 'Innerwall_' + c,
                'type' : 'InnerWall',
                'direction' : 'Vertical',
                'Layers' : get_inner_wall_layers(),
                'area' : areas_innerwall[c],
                'spaces' : correspondence[c]
            }])

    return iws

## 8.3. InnerWalls Element - 統合

#### Function

In [58]:
def convert_innerwall( d ):

    innerwalls =[]
    
    # 室と床下空間の間の床の推定
    innerwalls.extend(get_innerwall_groundfloor_dictionary(d))
    
    # 室間の床・天井の推定
    innerwalls.extend(get_innerwall_innerslab(d, innerwalls))
    
    # 内壁等の推定              
    innerwalls.extend(get_innerwall_wall_dictionary(d))
            
    return innerwalls

#### Example

In [59]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_layers()},
]

convert_innerwall( d )

[{'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
  'area': 6.666666666666667,
  'direction': 'Horizontal',
  'name': 'GroundFloor_other',
  'spaces': ['other', 'underfloor'],
  'type': 'GroundFloor'},
 {'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
  'area': 3.3333333333333335,
  'direction': 'Horizontal',
  'name': 'GroundFloor_nonliving',
  'spaces': ['nonliving', 'underfloor'],
  'type': 'GroundFloor'},
 {'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
  'area': 53.333333333333336,
  'direction': 'Horizontal',
  'name': 'InnerFloor_other',
  'nextspace': 'nonliving',
  'space': 'other',
  'type': 'InnerFloor'},
 {'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
  'area': 26.666666666666668,
  'direction': 'Horizontal',
  'name': 'InnerFloor_nonliving',
  'nextspace': 'other',
  'space': 'nonliving',
  'type': 'InnerFloor'},
 {'Layers': [{'cond': 0.16, 'name': 'PED', 'specH':

## 9. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

### Function

In [60]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('EarthFloors' in d) == True:
        d_lv4_input['EarthFloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

### Example

In [63]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'N', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 
          'WallConstructionMethod': 'WallInsuladdBackvertical',
          'Parts': [{'TypeWall': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeWall': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'RoofConstructionMethod': 'Insulrafter', 
          'Parts': [{'TypeRoof': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeRoof': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'CeilingConstructionMethod': 'Insulbeam', 
          'Parts': [{'TypeCeiling': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeCeiling': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUR',
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputUA', 
          'direction': 'top','area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'direction': 'top', 'IsInContactWithOutsideAir': True, 
          'area': 67.8, 'space': 'main', 'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Single', 
          'IsEtaValueInput': True, 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Double', 'IsEtaValueInput': False,
          'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Double', 'IsEtaValueInput': True,
          'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'space': 'main', 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'space': 'main', 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E', 'space': 'main' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W', 'space': 'main' }
    ],
    'EarthfloorPerimeters': [
        { 'name': 'NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' },
        { 'name': 'NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ]
}

convert(d)

{'Common': {'IsSimplifiedInput': False,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'IsSunshadeInput': False,
   'U': 6.51,
   'area': 2.52,
   'direction': 'NW',
   'name': 'DoorNW',
   'space': 'main'},
  {'IsSunshadeInput': True,
   'U': 4.65,
   'Y1': 0.0,
   'Y2': 1.0,
   'Z': 0.6,
   'area': 2.16,
   'direction': 'NE',
   'name': 'DoorNE',
   'space': 'main'}],
 'EarthfloorPerimeters': [{'direction': 'NW',
   'length': 2.43,
   'name': 'NW',
   'psi': 1.8,
   'space': 'underfloor'},
  {'direction': 'NE',
   'length': 2.43,
   'name': 'NE',
   'psi': 1.8,
   'space': 'underfloor'}],
 'Heatbridges': [{'direction1': 'N',
   'direction2': 'E',
   'length': 1.0,
   'name': 'NE',
   'psi': 1.8,
   'space': 'main',
   'structure': 'RC'},
  {'direction1': 'N',
   'direction2': 'W',
   'length': 2.0,
   'name': 'NW',
   'psi': 1.8,
   'space': 'main',
   'structure': 'Steel'}],
 'InnerWalls': [{'Layers':

### 〇) 室上の床面積・室下の天井面積の推定：案１

$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$は、既知とする。  
$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$は、各用途の床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の合計面積を、用途別床面積より差し引いて推定する。  
これらより、$A_{if,MR,OR}$,$A_{if,MR,NR}$,$A_{if,OR,NR}$,$A_{if,OR,NR}$,$A_{if,NR,MR}$,$A_{if,NR,OR}$を推定する。

各用途の室上の床面積は、その用途と他の2用途の間の室上の床面積の合計に等しい。

$$
\displaystyle
A_{if,MR} = A_{if,MR,OR}+A_{if,MR,NR}
$$
$$
\displaystyle
A_{if,OR} = A_{if,OR,MR}+A_{if,OR,NR}
$$
$$
\displaystyle
A_{if,NR} = A_{if,NR,MR}+A_{if,NR,OR}
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  

2用途の間の天井面積は、その2用途の間の床面積と一致するものとする。  

$$
\displaystyle
A_{ic,MR,OR} = A_{if,OR,MR}
$$
$$
\displaystyle
A_{ic,MR,NR} = A_{if,NR,MR}
$$
$$
\displaystyle
A_{ic,OR,MR} = A_{if,MR,OR}
$$
$$
\displaystyle
A_{ic,OR,NR} = A_{if,NR,OR}
$$
$$
\displaystyle
A_{ic,NR,MR} = A_{if,MR,NR}
$$
$$
\displaystyle
A_{ic,NR,OR} = A_{if,OR,NR}
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

各用途と他の2用途との間の室下の天井面積の比率は、他の2用途の室上の床面積の比率に一致すると仮定する。

$$
\displaystyle
A_{ic,MR,OR}:A_{ic,MR,NR} = A_{if,OR,MR}:A_{if,NR,MR} = A_{if,OR}:A_{if,NR}
$$
$$
\displaystyle
A_{ic,OR,MR}:A_{ic,OR,NR} = A_{if,MR,OR}:A_{if,NR,OR} = A_{if,MR}:A_{if,NR}
$$
$$
\displaystyle
A_{ic,NR,MR}:A_{ic,NR,OR} = A_{if,MR,NR}:A_{if,OR,NR} = A_{if,MR}:A_{if,OR}
$$

以上より、$A_{if,MR,OR}$,$A_{if,MR,NR}$,$A_{if,OR,NR}$,$A_{if,OR,NR}$,$A_{if,NR,MR}$,$A_{if,NR,OR}$が算出される。

$$
\displaystyle
A_{ic,MR,OR} = A_{if,MR,NR} = \frac{ A_{if,MR}}{2}
$$
$$
\displaystyle
A_{ic,OR,MR} = A_{if,OR,NR} = \frac{ A_{if,OR}}{2}
$$
$$
\displaystyle
A_{ic,NR,MR} = A_{if,NR,OR} = \frac{ A_{if,NR}}{2}
$$

$A_{if,NR,MR}$の算出の詳細は以下。他の2用途間についても同様。

$
\displaystyle
A_{if,MR,OR} = A_{if,MR} - A_{if,MR,NR}   
$  
$
\displaystyle
　　　　= A_{if,MR} - A_{if,OR,NR} \times \frac{A_{if,MR}}{A_{if,OR}}  
$  
$
\displaystyle
　　　　= A_{if,MR} - ( A_{if,OR} - A_{if,OR,MR}) \times \frac{A_{if,MR}}{A_{if,OR}}   
$  
$
\displaystyle
　　　　= A_{if,MR} - ( A_{if,OR} - A_{if,NR,MR} \times \frac{A_{if,OR}}{A_{if,NR}}) \times \frac{A_{if,MR}}{A_{if,OR}}  
$  
$
\displaystyle
　　　　= A_{if,NR,MR}\times \frac{A_{if,MR}}{A_{if,NR}}
$  
$
\displaystyle
A_{if,MR} : A_{if,NR} = A_{if,MR,OR} : A_{if,NR,OR} = A_{if,NR,MR}\times \frac{A_{if,MR}}{A_{if,NR}} : A_{if,NR} - A_{if,NR,MR}
$

$$
\displaystyle
∴　A_{if,NR,MR} = \frac{A_{if,NR}}{2}
$$

懸案事項  
　各用途の床面積（天井面積に等しいと想定）から、各用途の上向きの外皮の面積を差し引くと、各用途の室上の天井面積が推定される。  
　上記の方法で算出された各用途の室上の天井面積とは、乖離が生じ得る。

#### Function

In [ ]:
def get_area_floor_over_room_type1(A_if):
   
    return {'main to other'      : A_if['main']/2,
            'main to nonliving'  : A_if['main']/2,
            'other to main'      : A_if['other']/2,
            'other to nonliving' : A_if['other']/2,
            'nonliving to main'  : A_if['nonliving']/2,
            'nonliving to other' : A_if['nonliving']/2}

#### Example

In [ ]:
A_if = {'main': 6, 'other':3, 'nonliving': 6}

print(get_area_floor_over_room_type1(A_if))

### 〇) 室上の床面積・室下の天井面積の推定：案２

$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$,$A_{ic,MR,temp}$,$A_{ic,OR,temp}$,$A_{ic,NR,temp}$は既知とする。   
$A_{if,MR}$,$A_{if,OR}$,$A_{if,NR}$は、各用途の床下空間上の床と、方位が「下面」である外皮の部位と、土間床等の合計面積を、用途別床面積より差し引いて推定する。
$A_{ic,MR,temp}$,$A_{ic,OR,temp}$,$A_{ic,NR,temp}$は、各用途の方位が「上面」である外皮の部位の合計面積を、用途別床面積より差し引いて推定する。  
これらより、$A_{if,MR,OR}$,$A_{if,MR,NR}$,$A_{if,OR,NR}$,$A_{if,OR,NR}$,$A_{if,NR,MR}$,$A_{if,NR,OR}$を推定する。

各用途の室上の床面積は、その用途と他の2用途の間の室上の床面積の合計に等しい。

$$
\displaystyle
A_{if,MR} = A_{if,MR,OR}+A_{if,MR,NR}
$$
$$
\displaystyle
A_{if,OR} = A_{if,OR,MR}+A_{if,OR,NR}
$$
$$
\displaystyle
A_{if,NR} = A_{if,NR,MR}+A_{if,NR,OR}
$$

$A_{if,MR}$: その他の居室または非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR}$: 主たる居室または非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR}$: 主たる居室またはその他の居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,MR,OR}$: その他の居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,MR,NR}$: 非居室との間の、主たる居室の床の面積, m<sup>2</sup>  
$A_{if,OR,MR}$: 主たる居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,OR,NR}$: 非居室との間の、その他の居室の床の面積, m<sup>2</sup>  
$A_{if,NR,MR}$: 主たる居室との間の、非居室の床の面積, m<sup>2</sup>  
$A_{if,NR,OR}$: その他の居室との間の、非居室の床の面積, m<sup>2</sup>  

2用途の間の天井面積は、その2用途の間の床面積と一致するものとする。  

$$
\displaystyle
A_{ic,MR,OR} = A_{if,OR,MR}
$$
$$
\displaystyle
A_{ic,MR,NR} = A_{if,NR,MR}
$$
$$
\displaystyle
A_{ic,OR,MR} = A_{if,MR,OR}
$$
$$
\displaystyle
A_{ic,OR,NR} = A_{if,NR,OR}
$$
$$
\displaystyle
A_{ic,NR,MR} = A_{if,MR,NR}
$$
$$
\displaystyle
A_{ic,NR,OR} = A_{if,OR,NR}
$$

$A_{ic,MR,OR}$: その他の居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,MR,NR}$: 非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,MR}$: 主たる居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR,NR}$: 非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,MR}$: 主たる居室との間の、非居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR,OR}$: その他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

各用途の室上の床面積の合計、各用途の室下の天井面積の合計、各用途の室下の天井面積の暫定値の合計は、以下のとおり。

$$
\displaystyle
A_{if} = A_{if,MR} + A_{if,OR} + A_{if,NR}
$$
$$
\displaystyle
A_{ic,temp} = A_{ic,MR,temp} + A_{ic,OR,temp} + A_{ic,NR,temp}
$$

$A_{if}$: 室上の床の面積の合計, m<sup>2</sup>  
$A_{ic,MR,temp}$: その他の居室または非居室との間の、主たる居室の天井の面積の暫定値, m<sup>2</sup>  
$A_{ic,OR,temp}$: 主たる居室または非居室との間の、その他の居室の天井の面積の暫定値, m<sup>2</sup>  
$A_{ic,NR,temp}$: 主たる居室またはその他の居室との間の、非居室の天井の面積の暫定値, m<sup>2</sup>  

各用途の室下の天井面積は、その用途と他の2用途の間の室下の天井面積の合計に等しい。  
各用途の室下の天井面積は、他の2用途の室上の床面積以下である必要がある。

$$
\displaystyle
A_{ic,MR} = A_{ic,MR,OR}+A_{ic,MR,NR} = A_{if,OR,MR}+A_{if,NR,MR} ≦ A_{if,OR}+A_{if,NR}
$$
$$
\displaystyle
A_{ic,OR} = A_{ic,OR,MR}+A_{ic,OR,NR} = A_{if,MR,OR}+A_{if,NR,OR} ≦ A_{if,MR}+A_{if,NR}
$$
$$
\displaystyle
A_{ic,NR} = A_{ic,NR,MR}+A_{ic,NR,OR} = A_{if,MR,NR}+A_{if,OR,NR} ≦ A_{if,MR}+A_{if,OR}
$$

$A_{ic,MR}$: その他の居室または非居室との間の、主たる居室の天井の面積, m<sup>2</sup>  
$A_{ic,OR}$: 主たる居室または非居室との間の、その他の居室の天井の面積, m<sup>2</sup>  
$A_{ic,NR}$: 主たる居室またはその他の居室との間の、非居室の天井の面積, m<sup>2</sup>  

上記を満たすよう、$A_{ic,MR,temp}$,$A_{ic,OR,temp}$,$A_{ic,NR,temp}$を調整して、$A_{ic,MR}$,$A_{ic,OR}$,$A_{ic,NR}$を決定する。

$$
\displaystyle
A_{ic,MR} = \left \{
\begin{array}{ll}
A_{if,OR} + A_{if,NR} & ( A_{if} \times \frac{A_{ic,MR,temp}}{A_{ic,temp}} > A_{if,OR} + A_{if,NR} ) \\
A_{if,OR} \times \frac{A_{ic,MR,temp}}{A_{ic,MR,temp} + A_{ic,NR,temp}} & ( A_{if} \times \frac{A_{ic,OR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,NR} ) \\
A_{if,NR} \times \frac{A_{ic,MR,temp}}{A_{ic,MR,temp} + A_{ic,OR,temp}} & ( A_{if} \times \frac{A_{ic,NR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,OR} ) \\
A_{if} \times \frac{A_{ic,MR,temp}}{A_{ic,temp}} & ( 上記以外) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,OR} = \left \{
\begin{array}{ll}
A_{if,MR} \times \frac{A_{ic,OR,temp}}{A_{ic,OR,temp} + A_{ic,NR,temp}} & ( A_{if} \times \frac{A_{ic,MR,temp}}{A_{ic,temp}} > A_{if,OR} + A_{if,NR} ) \\
A_{if,MR} + A_{if,NR} & ( A_{if} \times \frac{A_{ic,OR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,NR} ) \\
A_{if,NR} \times \frac{A_{ic,OR,temp}}{A_{ic,MR,temp} + A_{ic,OR,temp}} & ( A_{if} \times \frac{A_{ic,NR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,OR} ) \\
A_{if} \times \frac{A_{ic,OR,temp}}{A_{ic,temp}} & ( 上記以外) \\
\end{array}
\right.
$$
$$
\displaystyle
A_{ic,NR} = \left \{
\begin{array}{ll}
A_{if,MR} \times \frac{A_{ic,NR,temp}}{A_{ic,OR,temp} + A_{ic,NR,temp}} & ( A_{if} \times \frac{A_{ic,MR,temp}}{A_{ic,temp}} > A_{if,OR} + A_{if,NR} ) \\
A_{if,OR} \times \frac{A_{ic,NR,temp}}{A_{ic,MR,temp} + A_{ic,NR,temp}} & ( A_{if} \times \frac{A_{ic,OR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,NR} ) \\
A_{if,MR} + A_{if,OR} & ( A_{if} \times \frac{A_{ic,NR,temp}}{A_{ic,temp}} > A_{if,MR} + A_{if,OR} ) \\
A_{if} \times \frac{A_{ic,NR,temp}}{A_{ic,temp}} & ( 上記以外) \\
\end{array}
\right.
$$

2用途間の床面積・天井面積が0㎡以上となるには、以下を満たす必要がある。

$$
\displaystyle
A_{if,MR,OR} ≧ max(0,A_{if,MR} - A_{if,MR,NR},A_{ic,OR} - A_{if,NR,OR}) ≧ max(0,A_{if,MR} - A_{ic,NR},A_{ic,OR} - A_{if,NR})
$$
$$
\displaystyle
A_{if,MR,NR} ≧ max(0,A_{if,MR} - A_{if,MR,OR},A_{ic,NR} - A_{if,OR,NR}) ≧ max(0,A_{if,MR} - A_{ic,OR},A_{ic,NR} - A_{if,OR})
$$
$$
\displaystyle
A_{if,OR,MR} ≧ max(0,A_{if,OR} - A_{if,OR,NR},A_{ic,MR} - A_{if,NR,MR}) ≧ max(0,A_{if,OR} - A_{ic,NR},A_{ic,MR} - A_{if,NR})
$$
$$
\displaystyle
A_{if,OR,NR} ≧ max(0,A_{if,OR} - A_{if,OR,MR},A_{ic,NR} - A_{if,MR,NR}) ≧ max(0,A_{if,OR} - A_{ic,MR},A_{ic,NR} - A_{if,MR})
$$$$
\displaystyle
A_{if,NR,MR} ≧ max(0,A_{if,NR} - A_{if,NR,OR},A_{ic,MR} - A_{if,OR,MR}) ≧ max(0,A_{if,NR} - A_{ic,OR},A_{ic,MR} - A_{if,OR})
$$
$$
\displaystyle
A_{if,NR,OR} ≧ max(0,A_{if,NR} - A_{if,NR,MR},A_{ic,OR} - A_{if,MR,OR}) ≧ max(0,A_{if,NR} - A_{ic,MR},A_{ic,OR} - A_{if,MR})
$$

上記を満たす場合に、隣接する用途が決定されない室上の床面積は、他の2用途に1/2ずつ分配する。

$$
\displaystyle
A_{if,MR,OR} = max(0,A_{if,MR} - A_{ic,NR},A_{ic,OR} - A_{if,NR}) + \frac{A_{if,MR} - max(0,A_{if,MR} - A_{ic,NR},A_{ic,OR} - A_{if,NR}) - max(0,A_{if,MR} - A_{ic,OR},A_{ic,NR} - A_{if,OR})}{2}
$$
$$
\displaystyle
A_{if,MR,NR} = max(0,A_{if,MR} - A_{ic,OR},A_{ic,NR} - A_{if,OR}) + \frac{A_{if,MR} - max(0,A_{if,MR} - A_{ic,NR},A_{ic,OR} - A_{if,NR}) - max(0,A_{if,MR} - A_{ic,OR},A_{ic,NR} - A_{if,OR})}{2}
$$
$$
\displaystyle
A_{if,OR,MR} = max(0,A_{if,OR} - A_{ic,NR},A_{ic,MR} - A_{if,NR})) + \frac{A_{if,OR} - max(0,A_{if,OR} - A_{ic,NR},A_{ic,MR} - A_{if,NR})) - max(0,A_{if,OR} - A_{ic,MR},A_{ic,NR} - A_{if,MR})}{2}
$$
$$
\displaystyle
A_{if,OR,NR} = max(0,A_{if,OR} - A_{ic,MR},A_{ic,NR} - A_{if,MR}) + \frac{A_{if,OR} - max(0,A_{if,OR} - A_{ic,NR},A_{ic,MR} - A_{if,NR})) - max(0,A_{if,OR} - A_{ic,MR},A_{ic,NR} - A_{if,MR})}{2}
$$$$
\displaystyle
A_{if,NR,MR} = max(0,A_{if,NR} - A_{ic,OR},A_{ic,MR} - A_{if,OR}) + \frac{A_{if,NR} - max(0,A_{if,NR} - A_{ic,OR},A_{ic,MR} - A_{if,OR}) - max(0,A_{if,NR} - A_{ic,MR},A_{ic,OR} - A_{if,MR}) }{2}
$$
$$
\displaystyle
A_{if,NR,OR} = max(0,A_{if,NR} - A_{ic,MR},A_{ic,OR} - A_{if,MR}) + \frac{A_{if,NR} - max(0,A_{if,NR} - A_{ic,OR},A_{ic,MR} - A_{if,OR}) - max(0,A_{if,NR} - A_{ic,MR},A_{ic,OR} - A_{if,MR}) }{2}
$$

懸案事項  
　推定方法が複雑である。

#### Function

In [ ]:
def get_area_floor_over_room_type2(A_if, A_ic_temp):
    
    A_if_sum = sum( A_if[x] for x in A_if.keys() )
    A_ic_temp_sum = sum( A_ic_temp[x] for x in A_ic_temp.keys() )
    
    if A_if_sum*A_ic_temp['main']/A_ic_temp_sum > A_if['other'] + A_if['nonliving'] :
        A_ic = {
            'main': A_if['other'] + A_if['nonliving'],
            'other': A_if['main']*A_ic_temp['other']/(A_ic_temp['other'] + A_ic_temp['nonliving']),
            'nonliving': A_if['main']*A_ic_temp['nonliving']/(A_ic_temp['other'] + A_ic_temp['nonliving'])
        }
    elif A_if_sum*A_ic_temp['other']/A_ic_temp_sum > A_if['main'] + A_if['nonliving'] :
        A_ic = {
            'main': A_if['other']*A_ic_temp['main']/(A_ic_temp['main'] + A_ic_temp['nonliving']),
            'other': A_if['main'] + A_if['nonliving'],
            'nonliving': A_if['other']*A_ic_temp['nonliving']/(A_ic_temp['main'] + A_ic_temp['nonliving'])
        }
    elif A_if_sum*A_ic_temp['nonliving']/A_ic_temp_sum > A_if['main'] + A_if['other'] :
        A_ic = {
            'main': A_if['nonliving']*A_ic_temp['main']/(A_ic_temp['main'] + A_ic_temp['other']),
            'other': A_if['nonliving']*A_ic_temp['other']/(A_ic_temp['main'] + A_ic_temp['other']),
            'nonliving': A_if['main'] + A_if['other']
        }
    else :
        A_ic = {
            'main': A_if_sum*A_ic_temp['main']/A_ic_temp_sum,
            'other': A_if_sum*A_ic_temp['other']/A_ic_temp_sum,
            'nonliving': A_if_sum*A_ic_temp['nonliving']/A_ic_temp_sum
        } 

    A_if_min = {
        'main to other'      : max(0, A_if['main'] - A_ic['nonliving'], A_ic['other'] - A_if['nonliving']),
        'main to nonliving'  : max(0, A_if['main'] - A_ic['other'], A_ic['nonliving'] - A_if['other']),
        'other to main'      : max(0, A_if['other'] - A_ic['nonliving'], A_ic['main'] - A_if['nonliving']),
        'other to nonliving' : max(0, A_if['other'] - A_ic['main'], A_ic['nonliving'] - A_if['main']),
        'nonliving to main'  : max(0, A_if['nonliving'] - A_ic['other'], A_ic['main'] - A_if['other']),
        'nonliving to other' : max(0, A_if['nonliving'] - A_ic['main'], A_ic['other'] - A_if['main'])
    }
   
    return {'main to other'      : A_if_min['main to other']
                                   + (A_if['main'] - A_if_min['main to other'] - A_if_min['main to nonliving'])/2,
            'main to nonliving'  : A_if_min['main to nonliving']
                                   + (A_if['main'] - A_if_min['main to other'] - A_if_min['main to nonliving'])/2,
            'other to main'      : A_if_min['other to main']
                                   + (A_if['other'] - A_if_min['other to main'] - A_if_min['other to nonliving'])/2,
            'other to nonliving' : A_if_min['other to nonliving']
                                   + (A_if['other'] - A_if_min['other to main'] - A_if_min['other to nonliving'])/2,
            'nonliving to main'  : A_if_min['nonliving to main']
                                   + (A_if['nonliving'] - A_if_min['nonliving to main'] - A_if_min['nonliving to other'])/2,
            'nonliving to other' : A_if_min['nonliving to other']
                                   + (A_if['nonliving'] - A_if_min['nonliving to main'] - A_if_min['nonliving to other'])/2}

#### Example

In [ ]:
A_if = {'main': 6, 'other':3, 'nonliving': 6}
A_ic_temp = {'main': 1, 'other':3, 'nonliving': 2}

print(get_area_floor_over_room_type2(A_if, A_ic_temp))